In [1]:
import pandas as pd

path_to_train = './data/train.csv'
path_to_test = './data/test.csv'

train = pd.read_csv(path_to_train, encoding='utf-8', usecols=['user_id', 'ts_listen', 'listen_type', 'is_listened'])
test = pd.read_csv(path_to_test, encoding='utf-8', usecols=['user_id', 'ts_listen', 'listen_type'])
test['is_listened'] = 0 # просто щоб зібрати, ми будемо використовувати попередні значення

df = pd.concat([train, test], ignore_index=True)
#df = train
df.tail()

,ts_listen,listen_type,user_id,is_listened
7578747,1479664431,1,12641,0
7578748,1480358099,1,10055,0
7578749,1480616853,1,1029,0
7578750,1479644510,1,4630,0
7578751,1480491885,1,6467,0


In [2]:
df = df.sort_values(by=['user_id', 'ts_listen'], axis=0)
df.tail()

,ts_listen,listen_type,user_id,is_listened
7575803,1478016326,1,19915,0
569368,1477512816,0,19916,1
7563826,1478017434,1,19916,0
4791398,1477926978,0,19917,1
7565747,1478009685,1,19917,0


In [3]:
df.head()

,ts_listen,listen_type,user_id,is_listened
682493,1477939775,0,0,1
4454939,1477982531,0,0,0
5354523,1477982561,0,0,0
5899458,1477982582,0,0,1
4716574,1477982792,0,0,1


In [4]:
df['cumsum_listen_type'] = (df['listen_type'].groupby((
                (df[['listen_type','user_id']].shift() != df[['listen_type','user_id']]).any(axis=1)
            ).cumsum()).cumcount() + 1).shift()

df['cumsum_listen_type_1'] = (df['listen_type'] * (df['listen_type'].groupby((
                (df[['listen_type','user_id']].shift() != df[['listen_type','user_id']]).any(axis=1)
            ).cumsum()).cumcount() + 1)).shift()

df['cumsum_listen_type_0'] = (abs(1-df['listen_type']) * (df['listen_type'].groupby((
                (df[['listen_type','user_id']].shift() != df[['listen_type','user_id']]).any(axis=1)
            ).cumsum()).cumcount() + 1)).shift()

fi = df.groupby('user_id', as_index=False).apply(lambda x: x.index[0])
df.ix[fi, 'cumsum_listen_type_1'] = 0
df.ix[fi, 'cumsum_listen_type_0'] = 0
df.ix[fi, 'cumsum_listen_type'] = 0

df[['user_id', 'listen_type', 'cumsum_listen_type', 'cumsum_listen_type_1', 'cumsum_listen_type_0']].tail(10)

,user_id,listen_type,cumsum_listen_type,cumsum_listen_type_1,cumsum_listen_type_0
2549309,19913,0,0.0,0.0,0.0
7575054,19913,1,1.0,0.0,1.0
2857394,19914,0,0.0,0.0,0.0
7573591,19914,1,1.0,0.0,1.0
4242361,19915,1,0.0,0.0,0.0
7575803,19915,1,1.0,1.0,0.0
569368,19916,0,0.0,0.0,0.0
7563826,19916,1,1.0,0.0,1.0
4791398,19917,0,0.0,0.0,0.0
7565747,19917,1,1.0,0.0,1.0


In [5]:
df[['user_id', 'listen_type', 'cumsum_listen_type', 'cumsum_listen_type_1', 'cumsum_listen_type_0']].head(10)

,user_id,listen_type,cumsum_listen_type,cumsum_listen_type_1,cumsum_listen_type_0
682493,0,0,0.0,0.0,0.0
4454939,0,0,1.0,0.0,1.0
5354523,0,0,2.0,0.0,2.0
5899458,0,0,3.0,0.0,3.0
4716574,0,0,4.0,0.0,4.0
2859238,0,0,5.0,0.0,5.0
3796053,0,0,6.0,0.0,6.0
1209271,0,0,7.0,0.0,7.0
6888438,0,0,8.0,0.0,8.0
6714892,0,0,9.0,0.0,9.0


In [6]:
df['cumsum_is_listened'] = (df['is_listened'].groupby((
                (df[['is_listened','user_id']].shift() != df[['is_listened','user_id']]).any(axis=1)
            ).cumsum()).cumcount() + 1).shift()

df['cumsum_is_listened_1'] = (df['is_listened'] * (df['is_listened'].groupby((
                (df[['is_listened','user_id']].shift() != df[['is_listened','user_id']]).any(axis=1)
            ).cumsum()).cumcount() + 1)).shift()

df['cumsum_is_listened_0'] = (abs(1-df['is_listened']) * (df['is_listened'].groupby((
                (df[['is_listened','user_id']].shift() != df[['is_listened','user_id']]).any(axis=1)
            ).cumsum()).cumcount() + 1)).shift()

fi = df.groupby('user_id', as_index=False).apply(lambda x: x.index[0])
df.ix[fi, 'cumsum_is_listened_1'] = 0
df.ix[fi, 'cumsum_is_listened_0'] = 0
df.ix[fi, 'cumsum_is_listened'] = 0

df[['user_id', 'is_listened', 'cumsum_is_listened', 'cumsum_is_listened_1', 'cumsum_is_listened_0']].tail(10)

,user_id,is_listened,cumsum_is_listened,cumsum_is_listened_1,cumsum_is_listened_0
2549309,19913,1,0.0,0.0,0.0
7575054,19913,0,1.0,1.0,0.0
2857394,19914,1,0.0,0.0,0.0
7573591,19914,0,1.0,1.0,0.0
4242361,19915,1,0.0,0.0,0.0
7575803,19915,0,1.0,1.0,0.0
569368,19916,1,0.0,0.0,0.0
7563826,19916,0,1.0,1.0,0.0
4791398,19917,1,0.0,0.0,0.0
7565747,19917,0,1.0,1.0,0.0


In [7]:
df[['user_id', 'is_listened', 'cumsum_is_listened', 'cumsum_is_listened_1', 'cumsum_is_listened_0']].head(10)

,user_id,is_listened,cumsum_is_listened,cumsum_is_listened_1,cumsum_is_listened_0
682493,0,1,0.0,0.0,0.0
4454939,0,0,1.0,1.0,0.0
5354523,0,0,1.0,0.0,1.0
5899458,0,1,2.0,0.0,2.0
4716574,0,1,1.0,1.0,0.0
2859238,0,1,2.0,2.0,0.0
3796053,0,1,3.0,3.0,0.0
1209271,0,1,4.0,4.0,0.0
6888438,0,1,5.0,5.0,0.0
6714892,0,1,6.0,6.0,0.0


In [8]:
user_count = df.groupby('user_id', as_index=False).agg({'ts_listen':'count'})
user_count['count'] = user_count['ts_listen']
del user_count['ts_listen']
df = df.join(user_count, on='user_id', rsuffix='_r')
df.head()

,ts_listen,listen_type,user_id,is_listened,cumsum_listen_type,cumsum_listen_type_1,cumsum_listen_type_0,cumsum_is_listened,cumsum_is_listened_1,cumsum_is_listened_0,user_id_r,count
682493,1477939775,0,0,1,0.0,0.0,0.0,0.0,0.0,0.0,0,6318
4454939,1477982531,0,0,0,1.0,0.0,1.0,1.0,1.0,0.0,0,6318
5354523,1477982561,0,0,0,2.0,0.0,2.0,1.0,0.0,1.0,0,6318
5899458,1477982582,0,0,1,3.0,0.0,3.0,2.0,0.0,2.0,0,6318
4716574,1477982792,0,0,1,4.0,0.0,4.0,1.0,1.0,0.0,0,6318


In [9]:
df['cumsum_is_listened'] = df['cumsum_is_listened']/df['count']
df['cumsum_is_listened_1'] = df['cumsum_is_listened_1']/df['count']
df['cumsum_is_listened_0'] = df['cumsum_is_listened_0']/df['count']

df.head(10)

,ts_listen,listen_type,user_id,is_listened,cumsum_listen_type,cumsum_listen_type_1,cumsum_listen_type_0,cumsum_is_listened,cumsum_is_listened_1,cumsum_is_listened_0,user_id_r,count
682493,1477939775,0,0,1,0.0,0.0,0.0,0.000000,0.000000,0.000000,0,6318
4454939,1477982531,0,0,0,1.0,0.0,1.0,0.000158,0.000158,0.000000,0,6318
5354523,1477982561,0,0,0,2.0,0.0,2.0,0.000158,0.000000,0.000158,0,6318
5899458,1477982582,0,0,1,3.0,0.0,3.0,0.000317,0.000000,0.000317,0,6318
4716574,1477982792,0,0,1,4.0,0.0,4.0,0.000158,0.000158,0.000000,0,6318
2859238,1477982978,0,0,1,5.0,0.0,5.0,0.000317,0.000317,0.000000,0,6318
3796053,1477983184,0,0,1,6.0,0.0,6.0,0.000475,0.000475,0.000000,0,6318
1209271,1477983362,0,0,1,7.0,0.0,7.0,0.000633,0.000633,0.000000,0,6318
6888438,1477983546,0,0,1,8.0,0.0,8.0,0.000791,0.000791,0.000000,0,6318
6714892,1477983730,0,0,1,9.0,0.0,9.0,0.000950,0.000950,0.000000,0,6318


In [10]:
del df['user_id_r']
del df['count']

In [11]:
df = df.sort_index()
df.tail()

,ts_listen,listen_type,user_id,is_listened,cumsum_listen_type,cumsum_listen_type_1,cumsum_listen_type_0,cumsum_is_listened,cumsum_is_listened_1,cumsum_is_listened_0
7578747,1479664431,1,12641,0,21.0,0.0,21.0,0.008621,0.008621,0.0
7578748,1480358099,1,10055,0,3.0,3.0,0.0,0.338462,0.338462,0.0
7578749,1480616853,1,1029,0,5.0,5.0,0.0,0.006029,0.006029,0.0
7578750,1479644510,1,4630,0,19.0,0.0,19.0,0.033962,0.033962,0.0
7578751,1480491885,1,6467,0,316.0,316.0,0.0,0.007958,0.007958,0.0


In [2]:
path_to_test_cumsum = './data/features/cumsum.test.csv'
path_to_train_cumsum = './data/features/cumsum.train.csv'
new_cols = ['cumsum_listen_type', 'cumsum_listen_type_1', 'cumsum_listen_type_0', 'cumsum_is_listened', 'cumsum_is_listened_1', 'cumsum_is_listened_0']

In [13]:
test_cumsum = df.iloc[range(len(train), len(train)+len(test))][new_cols]
test_cumsum.to_csv(path_to_test_cumsum, encoding='utf-8', index=False)

train_cumsum = df.iloc[range(len(train))][new_cols]
train_cumsum.to_csv(path_to_train_cumsum, encoding='utf-8', index=False)

-------------

In [3]:
import validation as v
import pandas as pd

path_to_train = './data/train.csv'

train = pd.read_csv(path_to_train, encoding='utf-8')
train_cumsum = pd.read_csv(path_to_train_cumsum, encoding='utf-8')

train = pd.concat([train, train_cumsum], axis=1)
train.head()

,genre_id,ts_listen,media_id,album_id,context_type,release_date,platform_name,platform_family,media_duration,listen_type,...,user_id,artist_id,user_age,is_listened,cumsum_listen_type,cumsum_listen_type_1,cumsum_listen_type_0,cumsum_is_listened,cumsum_is_listened_1,cumsum_is_listened_0
0,25471,1480597215,222606,41774,12,20040704,1,0,223,0,...,9241,55164,29,0,9.0,0.0,9.0,0.013216,0.000000,0.013216
1,25571,1480544735,250467,43941,0,20060301,2,1,171,0,...,16547,55830,30,1,4.0,0.0,4.0,0.024390,0.024390,0.000000
2,16,1479563953,305197,48078,1,20140714,2,1,149,1,...,7665,2704,29,1,2.0,0.0,2.0,0.356436,0.356436,0.000000
3,7,1480152098,900502,71521,0,20001030,0,0,240,0,...,1580,938,30,0,144.0,0.0,144.0,0.000929,0.000000,0.000929
4,7,1478368974,542335,71718,0,20080215,0,0,150,0,...,1812,2939,24,1,76.0,0.0,76.0,0.019019,0.019019,0.000000


In [1]:
path_to_folds = './data/lightgbm/cumsum'

In [5]:
v.generate_validation_folds(train, path_to_folds)

Fold 1


In [8]:
import model_lgbm as mlgbm

cols = [
    'genre_id', 'media_id', 'album_id', 
    'context_type',
    'release_date', 
    'platform_name', 
    'platform_family', 
    'media_duration',
    'listen_type',
    'user_gender', 
    'user_id', 'artist_id', 
    'user_age', 
    'cumsum_listen_type', 
    'cumsum_listen_type_1', 
    'cumsum_listen_type_0', 
    'cumsum_is_listened', 
    'cumsum_is_listened_1', 
    'cumsum_is_listened_0'
       ]

params = {
    'application':'binary',
    'num_leaves': 5,
    'max_depth': 3,
    'feature_fraction': 0.7,
    'bagging_fraction': 0.7,
    'max_bin': 5,
    'metric': 'auc',
    'verbose': 1,
    'lambda_l1': 1.2
}

In [ ]:
evs = mlgbm.crossvalidate_model(path_to_folds,cols,params,validate_on_both=True)

[1]	training's auc: 0.822282	valid_1's auc: 0.664244
[2]	training's auc: 0.828203	valid_1's auc: 0.666492
[3]	training's auc: 0.830174	valid_1's auc: 0.668255
[4]	training's auc: 0.836197	valid_1's auc: 0.672098
[5]	training's auc: 0.841818	valid_1's auc: 0.675275
[6]	training's auc: 0.842589	valid_1's auc: 0.677005
[7]	training's auc: 0.842559	valid_1's auc: 0.676954
[8]	training's auc: 0.84282	valid_1's auc: 0.675354
[9]	training's auc: 0.842964	valid_1's auc: 0.67625
[10]	training's auc: 0.84278	valid_1's auc: 0.676047
[11]	training's auc: 0.843609	valid_1's auc: 0.676368
[12]	training's auc: 0.844299	valid_1's auc: 0.669968
[13]	training's auc: 0.844246	valid_1's auc: 0.671183
[14]	training's auc: 0.845267	valid_1's auc: 0.674384
[15]	training's auc: 0.847103	valid_1's auc: 0.676728
[16]	training's auc: 0.846986	valid_1's auc: 0.676439
[17]	training's auc: 0.84796	valid_1's auc: 0.677786
[18]	training's auc: 0.847845	valid_1's auc: 0.677392
[19]	training's auc: 0.848695	valid_1's a

In [ ]:
path_to_sample = './data/sample_submission_kaggle.csv'
path_to_submission = './data/submission.csv'
path_to_test = './data/test.csv'

test = pd.read_csv(path_to_test, encoding='utf-8')
test = pd.concat([test, pd.read_csv(path_to_feature_test, encoding='utf-8')], axis=1)

test.head()

In [ ]:
fs = mlgbm.create_submission(train, test, path_to_sample, params, cols, path_to_submission)

In [ ]:
fscore = list(zip(cols, fs))
fscore.sort(key=lambda x: x[1])
fscore.reverse()
fscore

0            0.0
1            1.0
2          108.0
3            0.0
4           19.0
5           37.0
6            1.0
7            2.0
8            4.0
9          925.0
10          31.0
11          46.0
12           2.0
13           0.0
14           0.0
15          14.0
16          14.0
17          89.0
18           1.0
19           1.0
20          60.0
21          48.0
22          46.0
23          80.0
24           0.0
25          12.0
26          47.0
27          41.0
28           1.0
29           0.0
           ...  
7558804      0.0
7558805      0.0
7558806     48.0
7558807      0.0
7558808      2.0
7558809      7.0
7558810      3.0
7558811     11.0
7558812    112.0
7558813      1.0
7558814     53.0
7558815      2.0
7558816      0.0
7558817      2.0
7558818     99.0
7558819     18.0
7558820     37.0
7558821      0.0
7558822     81.0
7558823      0.0
7558824      6.0
7558825     16.0
7558826     17.0
7558827      5.0
7558828      0.0
7558829      0.0
7558830      1.0
7558831      0